In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Arsenic/P1011.2. Analyses/P1011.2.2. Arsenic Prediction/Output')
cur_date = "021324"

library(readxl)
library(openxlsx)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_020924.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.3     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.3     ✔ tibble  3.2.1
✔ purrr   1.0.2     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,Ar,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816,2.007062,ND
2,04009002C,1525,14-18,12/7/10,65,300,32,2.0,7.0,Ar,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912,1.912321,ND
3,04030015C,1525,14-18,8/27/12,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,7.000000,D
4,04030015J,1525,14-18,4/5/10,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,1.153921,ND
5,04030020H,234,09-147,10/25/10,52,125,36,20.0,7.6,Ar,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,1.675360,ND
6,4030041,1515,14-04,3/2/16,47,275,34,2.5,8.2,Ar,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955,14.000000,D


Using `water_sample_date`, `casing_depth`, `well_depth`, `static_water_depth`, `pH`, `flow_rate`, `Geologic_Terrane` or `Rock_Type`, `Soil_Type_Condensed`, `Landuse_Condensed`, `latitude`, and `longitude` to predict with As concentration falls above or below a certain threshold. RF and SVM models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. Limit of Detection (< or >= 5ppb)
3. EPA's Maximum Contaminant Level (MCL) (< or >= 10ppb)

Starting by creating 2 additional variables for above and below 5 and 10ppb and formatting the df for input into ML models.

In [3]:
arsenic_data = well_data %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration >= 5, 1, 0)), ref = "0"),
          Ten_Threshold = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
           # making this col into a factor
          Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date)) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar" & Landuse_Condensed != "NA") %>%
    select(-Metal)

head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  6 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longitude,⋯,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,-80.55427,⋯,CSB,MV,H,C,675.174,654.6816,2.007062,0,0,0
2,04009002C,1525,14-18,2010-12-07,65,300,32,2.0,7.0,-80.55676,⋯,CSB,MV,D,C,678.113,454.1912,1.912321,0,0,0
3,04030015C,1525,14-18,2012-08-27,65,300,32,2.0,8.1,-80.55676,⋯,CSB,MV,D,C,469.708,454.1912,7.000000,1,1,0
4,04030015J,1525,14-18,2010-04-05,65,300,32,2.0,8.1,-80.55676,⋯,CSB,MV,H,F,470.293,454.1912,1.153921,0,0,0
5,04030020H,234,09-147,2010-10-25,52,125,36,20.0,7.6,-80.55220,⋯,CSB,MV,H,F,470.293,918.3419,1.675360,0,0,0
6,4030041,1515,14-04,2016-03-02,47,275,34,2.5,8.2,-80.56423,⋯,CSB,MV,H,D,470.293,512.3955,14.000000,1,1,1


In [4]:
# original number of records
dim(well_data)

# records kept for analysis
dim(arsenic_data)

[1] 2145   20

[1] 714  21

# Variable Selection

To determine which variables are collinear and need to be removed, but first doing this with continuous data only to run Spearman's correlation tests. The continuous variables include: `casing_depth`, `well_depth`, `static_water_depth`, `pH`, `flow_rate`, `Elevation`, 
                    `Stream_Distance`,`latitude`, and `longitude`.

In [5]:
continuous_vars = c('Casing_Depth', 'Well_Depth', 'Static_Water_Depth', 'pH', 'Flow_Rate', 'Latitude', 'Longitude', 'Elevation', 
                    'Stream_Distance')

# creating correlation matrix
corr_matrix = rcorr(as.matrix(arsenic_data[,continuous_vars]), type = "spearman")
# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



Variable1,Variable2,Spearman Coefficient
<chr>,<fct>,<dbl>


No features passed the threshold, so none of the continuous variables are highly correlated to each other. Now using ANOVA tests for continuous and multi-categorical variable comparisons. The categorical variables include: `Geologic_Terrane`, `Rock_Type`, `Soil_Type_Condensed`, and `Landuse_Condensed`.

In [13]:
cateogorical_vars = c('Geology', 'Soil_Type', 'Landuse')
# t = aov(Longitude ~ Geologic_Terrane, data = arsenic_data)
# summary(t)

# aov_test = function(variable1, variable2){
#     t = aov(variable1 ~ variable2, data = arsenic_data)
#     output = summary(t)
#     return(output)
# }
# aov_test('Longitude', 'Geologic_Terrane')

Running correlations between categorical variables using a chi-square test.

In [14]:
get_tables = function(df, variable1, variable2){
    # creating tables for each variable
    overall_variable = df %>% 
        # need the group_by_at function to pass a function parameter into dplyr
        group_by_at(variable1) %>%
        select(all_of(variable2)) 

    variable_table = table(overall_variable)
    return(variable_table)
}

# calling fn
geology_soil_table = get_tables(arsenic_data, 'Geologic_Terrane', 'Soil_Type_Condensed')
geology_landuse_table = get_tables(arsenic_data, 'Geologic_Terrane', 'Landuse_Condensed')
landuse_soil_table = get_tables(arsenic_data, 'Landuse_Condensed', 'Soil_Type_Condensed')
rock_soil_table = get_tables(arsenic_data, 'Rock_Type', 'Soil_Type_Condensed')
rock_landuse_table = get_tables(arsenic_data, 'Rock_Type', 'Landuse_Condensed')

# geology_soil_table
# geology_landuse_table
# landuse_soil_table
# rock_soil_table
# rock_landuse_table

Adding missing grouping variables: `Geologic_Terrane`
Adding missing grouping variables: `Geologic_Terrane`
Adding missing grouping variables: `Landuse_Condensed`
Adding missing grouping variables: `Rock_Type`
Adding missing grouping variables: `Rock_Type`


In [9]:
# fisher exact test
fisher.test(geology_soil_table)
fisher.test(geology_landuse_table) 
# not enough data and threw errors, so I changed the simulate p value parameter
fisher.test(landuse_soil_table, simulate.p.value = TRUE)
fisher.test(rock_soil_table, simulate.p.value = TRUE)
fisher.test(rock_landuse_table, simulate.p.value = TRUE)


	Fisher's Exact Test for Count Data

data:  geology_soil_table
p-value = 4.508e-05
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  geology_landuse_table
p-value = 0.0004902
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  landuse_soil_table
p-value = 0.0004998
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  rock_soil_table
p-value = 0.0004998
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data with simulated p-value (based on
	2000 replicates)

data:  rock_landuse_table
p-value = 0.0004998
alternative hypothesis: two.sided


Based on this, all these variables are highly correlated to each other. Therefore, I'll exclude `Geologic_Terrane` and include `Rock_Type` or `Soil_Type`, but not both. exclude landuse too????

# Summary Statistics
Determining if there are any signficiant differenes between the predictor variables for each outcome variable.

In [27]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(4:17)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 517","**1**, N = 197",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,712,2013-06-05 (979.174260670888),2013-03-09 (958.168767022588),0.3
Casing_Depth,714,74 (33),55 (23),<0.001
Well_Depth,714,301 (144),333 (128),0.007
Static_Water_Depth,714,35 (12),36 (13),0.4
Flow_Rate,714,25 (33),14 (16),<0.001
pH,714,7.45 (0.54),7.82 (0.40),<0.001
Longitude,714,-80.67 (0.12),-80.51 (0.13),<0.001
Latitude,714,34.97 (0.09),35.01 (0.09),<0.001
Geologic_Terrane,714,NA,NA,<0.001


In [29]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:17)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 521","**1**, N = 193",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,712,2013-06-01 (979.73581125107),2013-03-16 (956.85311996601),0.3
Casing_Depth,714,74 (33),55 (23),<0.001
Well_Depth,714,300 (144),335 (128),0.003
Static_Water_Depth,714,35 (12),35 (13),0.6
Flow_Rate,714,25 (33),14 (16),<0.001
pH,714,7.44 (0.54),7.83 (0.40),<0.001
Longitude,714,-80.66 (0.12),-80.51 (0.13),<0.001
Latitude,714,34.97 (0.09),35.01 (0.09),<0.001
Geologic_Terrane,714,NA,NA,<0.001


In [30]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:17)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 582","**1**, N = 132",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,712,2013-05-13 (973.00384387001),2013-05-03 (979.450720926567),>0.9
Casing_Depth,714,73 (33),51 (19),<0.001
Well_Depth,714,306 (146),327 (109),0.11
Static_Water_Depth,714,35 (13),34 (11),0.4
Flow_Rate,714,24 (32),13 (15),<0.001
pH,714,7.48 (0.54),7.87 (0.41),<0.001
Longitude,714,-80.65 (0.13),-80.49 (0.11),<0.001
Latitude,714,34.97 (0.09),35.02 (0.09),<0.001
Geologic_Terrane,714,NA,NA,<0.001


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance. Class imbalance will be addressed using SMOTE. 

In [13]:
# creating dfs for each outcome
# removing landuse and geologic terrane
# dropped 2 rows that had missing dates
arsenic_detect_df = drop_na(arsenic_data[,c(10,11,13,19)]) 
arsenic_five_df = drop_na(arsenic_data[,c(10,11,13,20)]) 
arsenic_ten_df = drop_na(arsenic_data[,c(10,11,13,21)]) 

head(arsenic_detect_df)

,Longitude,Latitude,Rock_Type,Detect_Concentration
,<dbl>,<dbl>,<chr>,<fct>
1,-80.55427,34.87224,MV,0
2,-80.55676,34.87109,MV,0
3,-80.55676,34.87109,MV,1
4,-80.55676,34.87109,MV,0
5,-80.55220,34.86012,MV,0
6,-80.56423,34.88559,MV,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [14]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [15]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df, "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [16]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3),
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"), Kernel = NA,
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]])) %>%
    rename(`Balanced Accuracy` = Balanced.Accuracy)

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Outcome,Kernel,Balanced Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Detect_Concentration,NA,0.7593397,0.6966667,0.8220127,0.5983761,0.7593397
RF Classification,Five_Threshold,NA,0.7757018,0.7203779,0.8310256,0.6141329,0.7757018
RF Classification,Ten_Threshold,NA,0.7582695,0.6660969,0.8504421,0.5059265,0.7582695


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,235.13059
Latitude,142.72707
Rock_Type,22.59916


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,234.39247
Latitude,141.66354
Rock_Type,28.05664


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longitude,267.86802
Latitude,147.04689
Rock_Type,40.80087


Rerunning RF model with noise variables to determine, which predictors are significant above the background noise.

In [83]:
train_vars_noise_arsenic_detect = rf_values_arsenic_detect[[1]]
train_vars_noise_arsenic_five = rf_values_arsenic_five[[1]]
train_vars_noise_arsenic_ten = rf_values_arsenic_ten[[1]]

noise_df = function(train_vars_noise){
    set.seed(8)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[10])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[2])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[9])]], replace = TRUE)
    
    return(train_vars_noise)
}

# calling fn
noise_training_rf_dataset_arsenic_detect = noise_df(train_vars_noise_arsenic_detect)
noise_training_rf_dataset_arsenic_five = noise_df(train_vars_noise_arsenic_five)
noise_training_rf_dataset_arsenic_ten = noise_df(train_vars_noise_arsenic_ten)
head(noise_training_rf_dataset_arsenic_detect)

ERROR: Error in `[.data.frame`(train_vars_noise, 10): undefined columns selected


In [ ]:
# calling fn
noise_rf_values_arsenic_detect = rf_classification(noise_training_rf_dataset_arsenic_detect, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
noise_rf_values_arsenic_five = rf_classification(noise_training_rf_dataset_arsenic_five, "Five_Threshold", 
                                             "pred_Five_Threshold")
noise_rf_values_arsenic_ten = rf_classification(noise_training_rf_dataset_arsenic_ten, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

In [ ]:
# viewing results
noise_rf_confusion_matrix = data.frame(Model = rep(c("RF w/ Noise"), times = 3),
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(noise_rf_values_arsenic_detect[[2]], noise_rf_values_arsenic_five[[2]], 
                  noise_rf_values_arsenic_ten[[2]]))

noise_rf_confusion_matrix

Calculating the average noise rank to determine, which predictors ranked higher than the highest noise rank.

In [ ]:
noise_importance_values_detect = noise_rf_values_arsenic_detect[[3]] %>%
    column_to_rownames(var = "Predictor")
noise_importance_values_five = noise_rf_values_arsenic_five[[3]] %>%
    column_to_rownames(var = "Predictor")
noise_importance_values_ten = noise_rf_values_arsenic_ten[[3]] %>%
    column_to_rownames(var = "Predictor")

In [ ]:
highest_noise_rank = function(noise_importance_values){
    # this gives us values between 0 and 1
    # the closer the number gets to 1 the lower it ranks, so the number closest to 1 is the max rank
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    # figuring out which is the highest mean decrease gini or background noise importance value
    highest_noise_rank = min(noise1rank, noise2rank, noise3rank, noise4rank, noise5rank)
    
    return(highest_noise_rank)
}

# calling fn
highest_noise_rank_detect = highest_noise_rank(noise_importance_values_detect)
highest_noise_rank_five = highest_noise_rank(noise_importance_values_five)
highest_noise_rank_ten = highest_noise_rank(noise_importance_values_ten)
highest_noise_rank_detect
highest_noise_rank_five
highest_noise_rank_ten

In [ ]:
last_significant_rank = function(noise_importance_values, highest_noise_rank){
    for (i in 1:length(noise_importance_values$MeanDecreaseGini)){
    # if the row number (index)/# of variables is less than the highest noise rank the loop will continue
        if (i/length(noise_importance_values$MeanDecreaseGini) > highest_noise_rank){
            last_significant_var = i - 1
            break
        }
    }    
    return(last_significant_var)
}

# calling fn
last_significant_rank_detect = last_significant_rank(noise_importance_values_detect, highest_noise_rank_detect)
last_significant_rank_five = last_significant_rank(noise_importance_values_five, highest_noise_rank_five)
last_significant_rank_ten = last_significant_rank(noise_importance_values_ten, highest_noise_rank_ten)
last_significant_rank_detect
last_significant_rank_five
last_significant_rank_ten

In [ ]:
# now getting those significant predictors
significant_predictors_detect = rownames(noise_importance_values_detect)[1:last_significant_rank_detect]
significant_predictors_five = rownames(noise_importance_values_five)[1:last_significant_rank_five]
significant_predictors_ten = rownames(noise_importance_values_ten)[1:last_significant_rank_ten]

In [ ]:
# converting row names back to columns to make next step easier
noise_importance_values_detect = noise_importance_values_detect %>%
    rownames_to_column(var = "Predictor")
noise_importance_values_five = noise_importance_values_five %>%
    rownames_to_column(var = "Predictor")
noise_importance_values_ten = noise_importance_values_ten %>%
    rownames_to_column(var = "Predictor")

In [ ]:
# getting dataframe ready to export variable importance ranks from above and whether those features passed the 
# background filter
noise_variable_importance_ranks = function(noise_importance_values_df, significant_predictors, outcome, predictors){
    # ADD WORDS!!!!!!!
    
    # filtering the original rank df for predictors that were significant (above random noise)
    filtered_significant_ranks_df = noise_importance_values_df %>%
        filter(Predictor %in% significant_predictors) %>%
        arrange(-MeanDecreaseGini)
    
    # adding a column denoting if the feature was above random noise
    filtered_significant_ranks_df$Passed_Filter = c(rep(c("Yes"), 
                                                        times = length(filtered_significant_ranks_df$Predictor)))
    
    final_df = full_join(filtered_significant_ranks_df, noise_importance_values_df)
    
    # adding a col for outcome and the predictors used
    final_df$Outcome = outcome
    final_df$Predictors = predictors
    return(final_df)
}

# calling fn
significant_predictors_df_detect = noise_variable_importance_ranks(noise_importance_values_detect, 
                                                                   significant_predictors_detect, "Detect", "All")
significant_predictors_df_five = noise_variable_importance_ranks(noise_importance_values_five, 
                                                                 significant_predictors_five, "Five", "All")
significant_predictors_df_ten = noise_variable_importance_ranks(noise_importance_values_ten, 
                                                                significant_predictors_ten, "Ten", "All")

# creating 1 df
significant_predictors_df = rbind(significant_predictors_df_detect, significant_predictors_df_five,
                                 significant_predictors_df_ten)
head(significant_predictors_df)

# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [17]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc)) %>%
    mutate(Model = "SVM", Outcome = outcome, Kernel = model) 

  metrics = metrics[,c(6:8,1:5)]
  
  return(metrics)
}

In [18]:
#calling fn
svm_detect_linear = svm_classification(arsenic_detect_df, "linear", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_detect_radial = svm_classification(arsenic_detect_df, "radial", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_detect_polynomial = svm_classification(arsenic_detect_df, "polynomial", "Detect_Concentration", "pred_Detect_Concentration",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_linear = svm_classification(arsenic_five_df, "linear", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_radial = svm_classification(arsenic_five_df, "radial", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_five_polynomial = svm_classification(arsenic_five_df, "polynomial", "Five_Threshold", "pred_Five_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_linear = svm_classification(arsenic_ten_df, "linear", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_radial = svm_classification(arsenic_ten_df, "radial", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))
svm_ten_polynomial = svm_classification(arsenic_ten_df, "polynomial", "Ten_Threshold", "pred_Ten_Threshold",
                                            0.035, 1:5, seq(0,1,0.2))

ERROR: Error in predict.svm(model, if (!is.null(validation.x)) validation.x else if (useFormula) data[-train.ind[[sample]], : test data does not match model !


In [ ]:
# creating final df
final_df = rbind(rf_confusion_matrix,
                 svm_detect_linear, svm_detect_radial, svm_detect_polynomial, svm_five_linear, svm_five_radial,
               svm_five_polynomial, svm_ten_linear, svm_ten_radial, svm_ten_polynomial)

final_df

In [25]:
# exporting
write.xlsx(rf_confusion_matrix, paste0(Output,"/", "As_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(significant_predictors_df, paste0(Output,"/", "Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)